[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/plugins/02-plugin-structure.ipynb)

# Plugin Structure Deep Dive

**Learning Path**: Skills → **Plugins** → Orchestration  
**Level**: Intermediate  
**Time**: 30 minutes  
**Prerequisites**: [01-what-is-plugin](01-what-is-plugin.ipynb)

---

## What You'll Learn

1. ✅ **Directory structure** - Required and optional components
2. ✅ **plugin.json schema** - All required fields (6767-c)
3. ✅ **Component directories** - skills/, commands/, agents/, hooks/
4. ✅ **Validation rules** - Enterprise standards
5. ✅ **Interactive parsing** - Build a plugin validator
6. ✅ **Common mistakes** - How to avoid them

---

## The Anatomy of a Plugin

### Complete Directory Structure

```
my-plugin/                         ← Plugin root
├── .claude-plugin/                ← Metadata directory
│   └── plugin.json                ← ONLY file allowed here
├── skills/                        ← Optional: capabilities
│   ├── skill-one/
│   │   ├── SKILL.md               ← Skill definition
│   │   └── references/            ← Heavy data (on-demand)
│   └── skill-two/
│       └── SKILL.md
├── commands/                      ← Optional: slash commands
│   ├── quick-action.md
│   └── analyze.md
├── agents/                        ← Optional: subagents
│   └── specialist.md
├── hooks/                         ← Optional: event handlers
│   └── hooks.json
├── scripts/                       ← Optional: helper scripts
│   └── validate.sh
├── .mcp.json                      ← Optional: MCP config
├── .claudeignore                  ← Optional: exclude from context
├── README.md                      ← Required: documentation
└── LICENSE                        ← Required: license file
```

### Critical Rules (6767-c)

**MUST**:
- `.claude-plugin/` contains ONLY `plugin.json`
- Component directories at plugin root (NOT inside `.claude-plugin/`)
- Only create directories you use (no empty placeholders)
- All paths relative or use `${CLAUDE_PLUGIN_ROOT}`

**MUST NOT**:
- Put components inside `.claude-plugin/`
- Use absolute paths
- Hardcode secrets
- Commit `.env` files

In [ ]:
from pathlib import Path
import json

def create_plugin_structure(plugin_name, components=["skills", "commands"]):
    """
    Generate plugin directory structure.
    
    Args:
        plugin_name: kebab-case plugin name
        components: List of component dirs to create
    """
    structure = {
        plugin_name: {
            ".claude-plugin": {
                "plugin.json": "metadata"
            },
            "README.md": "documentation",
            "LICENSE": "MIT or other SPDX"
        }
    }
    
    # Add component directories
    for component in components:
        structure[plugin_name][f"{component}/"] = "component directory"
    
    def print_tree(d, prefix=""):
        items = list(d.items())
        for i, (key, value) in enumerate(items):
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{key}")
            
            if isinstance(value, dict):
                extension = "    " if is_last else "│   "
                print_tree(value, prefix + extension)
    
    print("PLUGIN DIRECTORY STRUCTURE")
    print("=" * 60)
    print_tree(structure)
    
    return structure

# Example: Create structure for testing plugin
create_plugin_structure("testing-toolkit", ["skills", "commands", "agents"])

## plugin.json Schema (Enterprise Required)

### Required Fields

```json
{
  "name": "my-plugin",              // REQUIRED: kebab-case, ^[a-z0-9-]+$
  "version": "1.0.0",               // REQUIRED: SemVer (MAJOR.MINOR.PATCH)
  "description": "...",             // REQUIRED: Brief explanation
  "author": {                       // REQUIRED: Author object
    "name": "Developer Name",       // REQUIRED: Full name
    "email": "dev@example.com"      // REQUIRED: Email
  },
  "license": "MIT",                 // REQUIRED: SPDX identifier
  "keywords": ["tag1", "tag2"]      // REQUIRED: Array of strings
}
```

### Optional Fields

```json
{
  "repository": "https://github.com/user/repo",  // Git URL
  "homepage": "https://example.com",             // Website
  "bugs": "https://github.com/user/repo/issues", // Issue tracker
  "category": "productivity",                    // Marketplace category
  "dependencies": {},                            // Future: plugin deps
  "mcpServers": {},                              // Inline MCP config
  "hooks": {}                                    // Inline hooks config
}
```

### Field Validation Rules

| Field | Rule | Error Code |
|-------|------|------------|
| `name` | `^[a-z0-9-]+$`, max 64 chars | `NAMING_001` |
| `version` | SemVer `\d+\.\d+\.\d+` | `PLUGIN_012` |
| `author.email` | Valid email format | `PLUGIN_005` |
| `license` | SPDX identifier | `PLUGIN_007` |
| `keywords` | Non-empty array | `PLUGIN_010` |

In [ ]:
import re
from typing import Dict, List, Tuple

def validate_plugin_json(plugin_data: dict) -> Tuple[List, List]:
    """
    Validate plugin.json against enterprise standards (6767-c).
    
    Returns:
        (errors, warnings) - Lists of (field, code, message) tuples
    """
    errors = []
    warnings = []
    
    # Required fields
    required = ["name", "version", "description", "author", "license", "keywords"]
    
    for field in required:
        if field not in plugin_data:
            errors.append((field, "PLUGIN_001", f"Missing required field: {field}"))
    
    # Validate name (kebab-case)
    if "name" in plugin_data:
        name = plugin_data["name"]
        if not re.match(r'^[a-z0-9-]+$', name):
            errors.append(("name", "NAMING_001", f"Name must be kebab-case: {name}"))
        if len(name) > 64:
            errors.append(("name", "NAMING_002", f"Name exceeds 64 chars: {len(name)}"))
    
    # Validate version (SemVer)
    if "version" in plugin_data:
        version = plugin_data["version"]
        if not re.match(r'^\d+\.\d+\.\d+$', version):
            errors.append(("version", "PLUGIN_012", f"Version must be SemVer (X.Y.Z): {version}"))
    
    # Validate author
    if "author" in plugin_data:
        author = plugin_data["author"]
        if isinstance(author, dict):
            if "name" not in author:
                errors.append(("author.name", "PLUGIN_004", "Missing author.name"))
            if "email" not in author:
                errors.append(("author.email", "PLUGIN_005", "Missing author.email"))
            elif not re.match(r'.+@.+\..+', author["email"]):
                errors.append(("author.email", "PLUGIN_005", f"Invalid email: {author['email']}"))
        else:
            errors.append(("author", "PLUGIN_003", "Author must be object with name/email"))
    
    # Validate license
    if "license" in plugin_data:
        license_val = plugin_data["license"]
        common_licenses = ["MIT", "Apache-2.0", "GPL-3.0", "BSD-3-Clause", "ISC"]
        if license_val not in common_licenses:
            warnings.append(("license", "PLUGIN_007", f"Uncommon license: {license_val}. Use SPDX identifier."))
    
    # Validate keywords
    if "keywords" in plugin_data:
        keywords = plugin_data["keywords"]
        if not isinstance(keywords, list) or len(keywords) == 0:
            errors.append(("keywords", "PLUGIN_010", "Keywords must be non-empty array"))
    
    return errors, warnings

# Test cases
test_plugins = [
    {
        "name": "test-plugin",
        "version": "1.0.0",
        "description": "Test plugin",
        "author": {"name": "John Doe", "email": "john@example.com"},
        "license": "MIT",
        "keywords": ["test"]
    },
    {
        "name": "Bad Plugin Name",  # ❌ Spaces
        "version": "1.0",  # ❌ Not SemVer
        "description": "Bad example",
        "author": "John Doe",  # ❌ Not object
        "license": "MIT"
    }
]

print("PLUGIN.JSON VALIDATION")
print("=" * 60)
for i, plugin in enumerate(test_plugins, 1):
    print(f"\nTest Case {i}:")
    errors, warnings = validate_plugin_json(plugin)
    
    if errors:
        for field, code, message in errors:
            print(f"  ❌ [{code}] {message}")
    if warnings:
        for field, code, message in warnings:
            print(f"  ⚠️ [{code}] {message}")
    if not errors and not warnings:
        print("  ✅ Valid!")

## Component Directories

### skills/ Directory

**Structure**:
```
skills/
├── skill-name-one/
│   ├── SKILL.md          ← Skill definition
│   └── references/       ← Optional: heavy data
└── skill-name-two/
    └── SKILL.md
```

**Rules**:
- Each skill in its own directory
- Directory name = skill name (kebab-case)
- SKILL.md required in each
- references/ for large tables/docs

---

### commands/ Directory

**Structure**:
```
commands/
├── quick-action.md       ← /quick-action
├── analyze.md            ← /analyze
└── deploy.md             ← /deploy
```

**Rules**:
- Filename (without .md) = slash command name
- Optional YAML frontmatter
- Body = prompt that expands on invocation

---

### agents/ Directory

**Structure**:
```
agents/
├── code-reviewer.md      ← Specialized agent
└── security-scanner.md   ← Another agent
```

**Rules**:
- Filename = agent name
- YAML frontmatter required
- Body = agent instructions

---

### hooks/ Directory

**Structure**:
```
hooks/
└── hooks.json            ← Event handler config
```

**Alternative**: Inline in plugin.json under `"hooks"` key

In [ ]:
import os
from pathlib import Path

def analyze_plugin_structure(plugin_path: str) -> dict:
    """
    Analyze a plugin's directory structure.
    
    Returns:
        dict with component counts and issues
    """
    plugin_root = Path(plugin_path)
    
    if not plugin_root.exists():
        return {"error": f"Plugin path not found: {plugin_path}"}
    
    # Check required files
    plugin_json = plugin_root / ".claude-plugin" / "plugin.json"
    readme = plugin_root / "README.md"
    
    analysis = {
        "path": str(plugin_root),
        "has_plugin_json": plugin_json.exists(),
        "has_readme": readme.exists(),
        "components": {},
        "issues": []
    }
    
    # Check component directories
    components = ["skills", "commands", "agents", "hooks"]
    
    for component in components:
        component_dir = plugin_root / component
        if component_dir.exists():
            if component == "skills":
                # Count skill directories
                skill_dirs = [d for d in component_dir.iterdir() 
                             if d.is_dir() and (d / "SKILL.md").exists()]
                analysis["components"][component] = len(skill_dirs)
            elif component == "commands":
                # Count .md files
                cmd_files = list(component_dir.glob("*.md"))
                analysis["components"][component] = len(cmd_files)
            elif component == "agents":
                # Count agent files
                agent_files = list(component_dir.glob("*.md"))
                analysis["components"][component] = len(agent_files)
            elif component == "hooks":
                hooks_json = component_dir / "hooks.json"
                analysis["components"][component] = 1 if hooks_json.exists() else 0
    
    # Check for issues
    if not analysis["has_plugin_json"]:
        analysis["issues"].append("Missing .claude-plugin/plugin.json")
    if not analysis["has_readme"]:
        analysis["issues"].append("Missing README.md")
    
    # Check for files inside .claude-plugin/ (should only be plugin.json)
    claude_plugin_dir = plugin_root / ".claude-plugin"
    if claude_plugin_dir.exists():
        extra_files = [f.name for f in claude_plugin_dir.iterdir() 
                      if f.name != "plugin.json"]
        if extra_files:
            analysis["issues"].append(f"Extra files in .claude-plugin/: {extra_files}")
    
    return analysis

# Analyze a real plugin
analysis = analyze_plugin_structure("plugins/productivity/001-jeremy-taskwarrior-integration")

print("PLUGIN STRUCTURE ANALYSIS")
print("=" * 60)
print(f"Path: {analysis.get('path', 'N/A')}")
print(f"\nRequired Files:")
print(f"  plugin.json: {'✅' if analysis.get('has_plugin_json') else '❌'}")
print(f"  README.md: {'✅' if analysis.get('has_readme') else '❌'}")

if analysis.get('components'):
    print(f"\nComponents:")
    for component, count in analysis['components'].items():
        print(f"  {component}/: {count}")

if analysis.get('issues'):
    print(f"\nIssues Found:")
    for issue in analysis['issues']:
        print(f"  ❌ {issue}")
else:
    print("\n✅ No issues found!")

## Common Mistakes and How to Fix Them

### Mistake 1: Components Inside .claude-plugin/

**Wrong**:
```
.claude-plugin/
├── plugin.json
└── skills/           ❌ Wrong location
    └── my-skill/
```

**Correct**:
```
.claude-plugin/
└── plugin.json       ✅ Only plugin.json here
skills/               ✅ At plugin root
└── my-skill/
```

---

### Mistake 2: Empty Component Directories

**Wrong**:
```
skills/     ❌ Empty directory
commands/   ❌ Empty directory
agents/     ❌ Empty directory
```

**Correct**:
```
skills/     ✅ Only if contains skills
└── skill-one/
```

**Rule**: Only create directories you use

---

### Mistake 3: Absolute Paths in Config

**Wrong**:
```json
{
  "mcpServers": {
    "my-server": {
      "command": "/home/user/server.js"  ❌
    }
  }
}
```

**Correct**:
```json
{
  "mcpServers": {
    "my-server": {
      "command": "${CLAUDE_PLUGIN_ROOT}/server.js"  ✅
    }
  }
}
```

---

### Mistake 4: Hardcoded Secrets

**Wrong**:
```json
{
  "mcpServers": {
    "api": {
      "env": {
        "API_KEY": "sk_live_123..."  ❌ Hardcoded
      }
    }
  }
}
```

**Correct**:
```json
{
  "mcpServers": {
    "api": {
      "env": {
        "API_KEY": "${MY_API_KEY}"  ✅ Environment variable
      }
    }
  }
}
```

In [ ]:
def check_common_mistakes(plugin_path: str) -> List[str]:
    """
    Check for common plugin structure mistakes.
    """
    issues = []
    plugin_root = Path(plugin_path)
    
    if not plugin_root.exists():
        return [f"Plugin path not found: {plugin_path}"]
    
    # Mistake 1: Components inside .claude-plugin/
    claude_plugin_dir = plugin_root / ".claude-plugin"
    if claude_plugin_dir.exists():
        for item in claude_plugin_dir.iterdir():
            if item.name != "plugin.json":
                issues.append(f"Component '{item.name}' should be at plugin root, not in .claude-plugin/")
    
    # Mistake 2: Empty component directories
    for component in ["skills", "commands", "agents", "hooks"]:
        component_dir = plugin_root / component
        if component_dir.exists() and component_dir.is_dir():
            if not any(component_dir.iterdir()):
                issues.append(f"Empty directory: {component}/. Remove if unused.")
    
    # Mistake 3: Check plugin.json for absolute paths
    plugin_json_path = plugin_root / ".claude-plugin" / "plugin.json"
    if plugin_json_path.exists():
        with open(plugin_json_path) as f:
            content = f.read()
            if re.search(r'["\']/(home|Users)/', content):
                issues.append("Absolute paths detected in plugin.json. Use ${CLAUDE_PLUGIN_ROOT}.")
    
    return issues

# Check a plugin
issues = check_common_mistakes("plugins/productivity/001-jeremy-taskwarrior-integration")

print("COMMON MISTAKES CHECK")
print("=" * 60)
if issues:
    print("Issues found:")
    for issue in issues:
        print(f"  ❌ {issue}")
else:
    print("✅ No common mistakes found!")

## Key Takeaways

### What You Learned

1. ✅ **Directory structure** - .claude-plugin/ + components at root
2. ✅ **plugin.json schema** - 6 required fields (name, version, description, author, license, keywords)
3. ✅ **Component directories** - skills/, commands/, agents/, hooks/
4. ✅ **Validation** - Built validator for plugin.json
5. ✅ **Common mistakes** - Where components go, empty dirs, paths, secrets
6. ✅ **Analysis tools** - Interactive structure inspection

### Enterprise Requirements (6767-c)

- [ ] `.claude-plugin/` contains ONLY `plugin.json`
- [ ] Components at plugin root (not inside `.claude-plugin/`)
- [ ] Only create directories you use
- [ ] Name is kebab-case
- [ ] Version is SemVer
- [ ] Author has name + email
- [ ] No absolute paths
- [ ] No hardcoded secrets
- [ ] README.md present
- [ ] LICENSE file present

---

## Next Steps

1. **[03-build-your-first-plugin.ipynb](03-build-your-first-plugin.ipynb)** - Hands-on creation
2. **[04-mcp-server-plugins.ipynb](04-mcp-server-plugins.ipynb)** - Advanced MCP
3. **Practice**: Analyze existing plugins in marketplace

---

*Enterprise Standards Compliant • 6767-c • Version 1.0.0 • MIT License*